# 信息导论课 Lecture 4: 近似算法与编程思想 学习笔记

这份笔记根据 Ana Bell of MIT 的幻灯片内容进行整理、讲解和代码实战。内容分为两大块：
1.  **算法优化之旅：** 从“猜-查”法的局限性出发，引入“近似算法”，并最终进化到高效的“二分搜索”。
2.  **编程的核心思想：** 探讨超越具体语法的两大支柱——“分解”(Decomposition) 与“抽象”(Abstraction)，并介绍实现它们的基本工具——“函数”(Functions)。

---
## 第一部分：寻找答案的更好方式

### 1. 从“猜-查”法到“近似算法”

**幻灯片核心内容 (Slides 4-8):**

*   **“猜-查”法 (Guess-and-Check / Exhaustive Enumeration) 的局限：**
    *   它只能处理**答案可能性有限**的情况（比如找整数的平方根）。
    *   当答案的可能性是**无限**的（比如，任何浮点数的平方根），或者解空间太大时，“猜-查”法就行不通了。我们不可能测试每一个可能的值。

*   **引入“近似算法” (Approximation Methods):**
    *   **核心思想：** 放弃寻找“精确解”，转而寻找一个**“足够好”的解 (good enough answer)**。
    *   **如何定义“足够好”？** 我们引入一个参数 `epsilon` (ε)，它代表了我们能容忍的误差范围。只要我们找到的答案 `r` 满足 `|r² - x| < ε`，我们就认为找到了一个可接受的近似解。
    *   **基本算法：**
        1.  从一个猜测值 `guess` 开始（比如0）。
        2.  以一个非常小的步长 `increment` 递增 `guess`。
        3.  在每一步，检查 `guess²` 是否已经足够接近 `x`（即 `abs(guess**2 - x) < epsilon`）。
        4.  如果足够接近，就停止；否则，继续增加 `guess`。

*   **参数的权衡 (Trade-off):**
    *   **`increment` (步长):** 步长越小，结果越精确，但程序运行越慢。步长越大，速度越快，但可能“跨过”正确答案。
    *   **`epsilon` (误差):** ε 越小，要求的精度越高，程序运行越慢。ε 越大，精度越低，但速度越快。

### 2. 近似算法的陷阱：“过冲”问题 (Overshooting)

**幻灯片核心内容 (Slides 9-18):**

近似算法看起来很美，但有一个致命的陷阱。

*   **问题描述：** 在 `while abs(guess**2 - x) >= epsilon:` 这个循环中，如果我们的步长 `increment` 设置得不够小，`guess` 的平方可能会从一个“比目标小太多”的值，**直接跳到**一个“比目标大太多”的值，从而**完美地“跨过”了 `epsilon` 定义的那个“足够好”的区间**。
*   **后果：** 如果循环条件仅仅是检查 `guess` 是否“足够接近”，那么这个循环可能**永远不会停止**！因为它永远也无法进入那个“足够好”的区间。
*   **解决方案：** 增加一个额外的停止条件。我们不仅要检查是否“足够接近”，还要确保我们的猜测没有“跑得太远”。一个简单的保护性条件是 `guess**2 <= x`。
    *   **`while abs(guess**2 - x) >= epsilon and guess**2 <= x:`**
    *   这个新的循环条件意味着：“只要我的答案还不够好，**并且**我的猜测的平方还没超过目标值，我就继续猜。”
    *   一旦 `guess**2` 超过了 `x`，循环就会停止，避免了无限循环。此时我们可以检查，是因为找到了答案而停止，还是因为“过冲”而停止。

**拓展思考：**
这个“过冲”问题深刻地揭示了**处理浮点数和连续值时的编程复杂性**。我们永远不能用 `==` 来直接比较两个浮点数是否相等，而必须使用 `abs(a - b) < epsilon` 这种检查误差的方式。

In [ ]:
# --- 近似算法求平方根的实现与修正 ---

# 幻灯片中的初始版本 (可能存在无限循环问题)
def approximate_sqrt_v1(x):
    epsilon = 0.01
    increment = 0.0001
    guess = 0.0
    num_guesses = 0
    
    # 这个循环在某些情况下可能不会停止
    # 比如 increment 太大，导致 guess**2 直接跳过了 epsilon 区间
    while abs(guess**2 - x) >= epsilon:
        if guess**2 > x: # 加一个保护，防止猜得太大导致死循环
            print("Warning: Overshot the answer!")
            break
        guess += increment
        num_guesses += 1
        
    print(f"--- V1 for x = {x} ---")
    if abs(guess**2 - x) < epsilon:
        print(f"在 {num_guesses} 次猜测后找到近似解: {guess}")
    else:
        print(f"在 {num_guesses} 次猜测后失败")

# 幻灯片中修正后的版本 (更健壮)
def approximate_sqrt_v2(x):
    epsilon = 0.01
    increment = 0.0001
    guess = 0.0
    num_guesses = 0
    
    # 增加了 guess**2 <= x 这个条件来防止“过冲”
    while abs(guess**2 - x) >= epsilon and guess**2 <= x:
        guess += increment
        num_guesses += 1
        
    print(f"\n--- V2 for x = {x} ---")
    print(f"循环在 {num_guesses} 次后停止。")
    # 循环结束后，需要再次检查是因为找到了答案还是因为过冲
    if abs(guess**2 - x) < epsilon:
        print(f"找到近似解: {guess}")
    else:
        print("失败：可能因为步长太大而过冲了答案。")

# --- 测试 ---
# 一个容易成功的例子
approximate_sqrt_v2(24)

# 一个非常耗时，但最终能成功的例子
# 注意：这个例子在 Jupyter 中可能会运行很久，你可以手动停止它
# x = 54321
# approximate_sqrt_v2(x) 
print("\n对于 x=54321，近似法非常慢，我们将跳过它，直接看二分搜索的威力。")

### 3. 质的飞跃：二分搜索 (Bisection Search)

**幻灯片核心内容 (Slides 20-28):**

近似算法虽然可行，但它“一步一步走”的方式效率太低，是一种**线性复杂度**的搜索。有没有更聪明的方法？—— **二分搜索**。

*   **核心思想：** 不再从头开始一步步猜，而是在一个确定的**搜索区间 `[low, high]`** 内，**永远只猜这个区间的中点**。
    1.  **初始区间：** 对于求平方根问题，我们知道答案一定在 `[0, x]` 之间 (暂时假设 `x >= 1`)。
    2.  **猜测中点：** `guess = (low + high) / 2`。
    3.  **判断与缩小区间：**
        *   如果 `guess²` **太小**了 (小于 `x`)，说明真正的答案一定在 `guess` 的右边。因此，我们把搜索区间的**下界 `low` 移动到 `guess` 的位置**。新的搜索区间是 `[guess, high]`。
        *   如果 `guess²` **太大**了 (大于 `x`)，说明真正的答案一定在 `guess` 的左边。因此，我们把搜索区间的**上界 `high` 移动到 `guess` 的位置**。新的搜索区间是 `[low, guess]`。
    4.  **重复：** 在新的、更小的区间里，继续重复第2和第3步，直到找到足够好的答案。

*   **巨大优势：** 每一次猜测，我们都能将搜索范围**缩小一半**！
    *   **线性搜索 (近似法):** 每次只排除了 `increment` 这么一小个可能性。
    *   **二分搜索:** 每次都排除了**一半**的可能性。
    *   这使得算法的复杂度从**线性 (Linear)** 降到了**对数 (Logarithmic)**，效率得到了指数级的提升！

**拓展思考：二分搜索的适用条件 (Slide 28)**
二分搜索并非万能。它能被使用的前提是：
1.  **有序性 (Order):** 答案的搜索空间必须是有序的。
2.  **可判断性 (Comparison):** 对于任何一个猜测，我们必须能明确地判断出它“太大了”还是“太小了”。

In [ ]:
# --- 二分搜索求平方根的实现 ---

def bisection_sqrt(x):
    if x < 0:
        return "不能对负数开方"
    
    epsilon = 0.01
    num_guesses = 0
    
    # 幻灯片中的一个重要拓展 (Slide 38-39):
    # 需要根据 x 的大小来确定初始搜索范围
    if x < 1:
        low = x
        high = 1
    else:
        low = 0
        high = x
        
    guess = (high + low) / 2.0

    while abs(guess**2 - x) >= epsilon:
        # 核心判断逻辑
        if guess**2 < x:
            # 猜得太小了，答案在右半边
            low = guess
        else:
            # 猜得太大了，答案在左半边
            high = guess
        
        # 更新猜测值为新区间的中点
        guess = (high + low) / 2.0
        num_guesses += 1
        
    print(f"\n--- 二分搜索 for x = {x} ---")
    print(f"在 {num_guesses} 次猜测后找到近似解: {guess}")

# --- 测试 ---
bisection_sqrt(24)
bisection_sqrt(54321) # 对比近似法，这个几乎是瞬间完成！

# 测试 0 < x < 1 的情况
bisection_sqrt(0.5)

### 4. 终极加速：牛顿-拉弗森法 (Newton-Raphson)

**幻灯片核心内容 (Slides 41-44):**

还有一个比二分搜索收敛更快的算法，它利用了微积分的思想。

*   **核心思想：**
    1.  将求“k的平方根”问题，转化为求函数 `p(x) = x² - k` 的**根 (root)**，即 `p(x) = 0` 的解。
    2.  从一个初始猜测值 `g` 开始。
    3.  画出函数在 `(g, p(g))` 这一点的**切线 (tangent line)**。
    4.  这条切线与 x 轴的交点，通常会比 `g` 更接近真正的根。我们就把这个交点作为**下一次的猜测值**。
    5.  重复这个过程。

*   **迭代公式：**
    这个过程可以用一个简单的迭代公式来表示：
    `new_guess = g - p(g) / p'(g)`
    其中 `p'(g)` 是函数 `p(x)` 在 `g` 点的导数 (derivative)。
    对于 `p(x) = x² - k`，它的导数是 `p'(x) = 2x`。
    所以，求平方根的迭代公式就是：
    `new_guess = g - (g² - k) / (2g)`

*   **优势：** 牛顿法通常比二分搜索收敛得更快，尤其是在猜测值已经比较接近真值的时候。

In [ ]:
# --- 牛顿法求平方根的实现 ---

def newton_raphson_sqrt(k):
    if k < 0:
        return "不能对负数开方"
        
    epsilon = 0.01
    # 初始猜测值
    guess = k / 2.0
    num_guesses = 0
    
    while abs(guess*guess - k) >= epsilon:
        # 应用牛顿法的迭代公式
        guess = guess - (guess**2 - k) / (2 * guess)
        num_guesses += 1
        
    print(f"\n--- 牛顿法 for k = {k} ---")
    print(f"在 {num_guesses} 次猜测后找到近似解: {guess}")

# --- 测试 ---
newton_raphson_sqrt(24)
newton_raphson_sqrt(54321)

# 对比三种方法的猜测次数：
# - 近似法 (x=24): ~2450次
# - 二分法 (x=24): 9次
# - 牛顿法 (x=24): 4次
# 效率的巨大差异一目了然！


## 第二部分：超越语法，编程的道与术

**幻灯片核心内容 (Slides 46-57):**

在掌握了循环、分支等基本语法后，我们开始接触两个最重要的编程思想。

### 1. 分解 (Decomposition)
*   **是什么：** 将一个复杂的大问题，拆分成多个独立的、功能单一的、可组合的小问题。
*   **为什么：**
    *   **降低认知负担：** 人类的大脑不擅长同时处理太多复杂信息。将问题分解，可以让我们一次只专注于一个简单的小任务。
    *   **便于复用 (Reuse):** 一个被分解出来的、功能单一的模块（比如一个“计算圆面积”的模块），可以在其他很多程序中被重复使用。

### 2. 抽象 (Abstraction)
*   **是什么：** 忽略不必要的细节，专注于问题的核心功能。即**分离“做什么 (what)”和“怎么做 (how)”**。
*   **为什么：**
    *   **隐藏复杂性：** 当我们调用一个函数时，我们只需要知道它的**功能**（比如 `sqrt(x)` 能求平方根）和**接口**（需要传入一个数字 `x`），而**不需要**关心它内部是用二分法还是牛顿法实现的。
    *   **使代码易于理解和维护：** 我们可以用一个简单的名字（比如函数名 `calculate_circle_area`）来代表一整块复杂的代码逻辑，让主程序看起来更清晰。

### 3. 函数 (Functions)：实现分解与抽象的工具

函数就是实现上述两大思想的基本工具。
*   **定义函数 (Defining):** 使用 `def` 关键字，将一段代码逻辑打包，并给它一个名字。这就是**分解**。
*   **调用函数 (Calling):** 通过函数名来使用这段代码，只需提供输入（参数），即可获得输出（返回值）。调用者无需关心其内部实现。这就是**抽象**。

一个设计良好的函数应该具备：
*   **名称 (Name):** 清晰地描述其功能。
*   **参数 (Parameters):** 定义了它需要哪些输入。
*   **文档字符串 (Docstring):** 详细说明函数的功能、输入和输出，这是函数的使用“说明书”。
*   **函数体 (Body):** 具体的实现逻辑。
*   **返回值 (Return):** 通过 `return` 关键字，将计算结果返回给调用者。

In [ ]:
# --- 函数：分解与抽象的实践 ---

def is_even(i):
    """
    一个简单的函数示例，体现了分解与抽象。
    
    Specification (说明/文档字符串):
      Input: i, a positive integer
      Returns: True if i is even, otherwise False
    """
    # 函数体 (Body): 实现细节
    # print("正在检查数字", i) # 这是实现细节，调用者不需要知道
    remainder = i % 2
    return remainder == 0 # 返回值 (Return)

# --- 调用函数 ---

# 调用者 (user of the function)
# 1. 不关心 is_even 内部是如何实现的（是用的 % 还是别的什么方法）。(抽象)
# 2. 只需要知道它的名字、需要一个整数输入，并会返回一个布尔值。
# 3. 这段代码被分解出来，可以在任何需要判断奇偶数的地方复用。(分解)
print(f"10 是偶数吗? {is_even(10)}")
print(f"7 是偶数吗? {is_even(7)}")

# 我们可以通过 help() 函数查看文档字符串
help(is_even)